In [1]:
# Previous experiments:
# server: 612
# client: 614
# udp_sd_irqb --> first experiments run on startup.
# udp_sdfn_irqb --> change hashing algorithm to include ports.
# udp_sdfn_noirqb --> now turn off irqbalance.
# udp_sdfn_irqb --> ...and restore irqbalance.

# Multiserver experiments:
# s610 --> 32 cores
# c609 --> 64 cores

In [2]:
import os
import re
import pandas as pd
import numpy as np

import plotly
import plotly.plotly as py
from plotly.graph_objs import *

plotly.tools.set_credentials_file(username='theano', api_key='hkYRYxjyM0aAGA5nmGq1')

In [3]:
import matplotlib.pyplot as plt

import matplotlib
matplotlib.style.use('ggplot')

In [4]:
home = os.getcwd()
print(home)

/home/theano/mountpoint/tsch/mininetpipe/results/multiserv2


In [36]:
def load_datfile(fpath, proto):
    # Returns a dataframe
    df = pd.read_csv(fpath, 
                     index_col=False,
                     names=["ncli",proto])
    return df

def aggregate_multiserv(fpath, agg_fpath, nservers=32):
    # Writes a results file to the same directory
    df = pd.read_csv(fpath,
                    index_col=False,
                    names=["ncli",""])
    gp = df.groupby(np.arange(len(df)) // nservers)
    
    if len(gp.size().unique()) > 1:
        print("Warning: something wrong with this DF")
    
    summed = gp.sum()
    summed.to_csv(agg_fpath,
                  mode="a",
                  header=False,
                  index=False)
    os.rename(fpath, fpath + "_agged")

In [37]:
results_files = [
                 # "udp_sd_irqb.txt",
                 "udp_sdfn_irqb.txt", # --> change hashing algorithm to include ports.
                 # "udp_sd_noirqb.txt", # --> default hash, turn off irqbalance
                 # "tcp_noirqb.txt",
                 "tcp_irqb.txt",
                 # "tcp_sdfn_s605c610.txt",
                 # "udp_allcores_irqb_sdfn.txt",
                 # "tcp_allcores_irqb.txt",
                 "tcp_32_servers.txt",
                ]

In [44]:
#for fname in ["udp_4.txt","udp_5.txt"]:
#    aggregate_multiserv(os.path.join(home, fname), "udp_32_servers.txt")
    
for fname in ["tcp_3.txt"]:
    aggregate_multiserv(os.path.join(home, fname), "tcp_32_servers.txt")

In [45]:
gps = []

for fname in results_files:
    df = load_datfile(os.path.join(home, fname), fname)
    gps.append(df.groupby("ncli"))

In [46]:
fig = {
    'data': [
        {
            'x': gp.mean().index,
            'y': gp.mean()[gp.mean().columns[0]],
            'name': gp.mean().columns[0],
            'error_y': dict(
                type='data',
                array=gp.std()[gp.std().columns[0]],
                visible=True
            ),
        } for gp in gps
    ],
    'layout': {
        'xaxis': {'title': '# clients', 'type': 'log'},
        'yaxis': {'title': 'throughput (pps)'},
        'title': 'Small-packet throughput'
    }
}

py.iplot(fig, filename="somefilename")

In [18]:
# This is just in case I want to use plt; identical in what gets plotted
# to the plotly script

"""
fig, ax = plt.subplots(figsize=(8,6))

for gp in gps:
    gp.mean().plot(ax=ax, 
                   yerr=gp.std(),
                  )

ax.title.set_text("Small-packet throughput")

ax.set_xlabel("# clients")
ax.set_ylabel("throughput (pps)")

ax.set_xscale('log', basex=2)
#ax.set_yscale('log', basey=10)
        
ax.legend(loc=2)

plt.show()
"""

'\nfig, ax = plt.subplots(figsize=(8,6))\n\nfor gp in gps:\n    gp.mean().plot(ax=ax, \n                   yerr=gp.std(),\n                  )\n\nax.title.set_text("Small-packet throughput")\n\nax.set_xlabel("# clients")\nax.set_ylabel("throughput (pps)")\n\nax.set_xscale(\'log\', basex=2)\n#ax.set_yscale(\'log\', basey=10)\n        \nax.legend(loc=2)\n\nplt.show()\n'